# SVM

In [1]:
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

In [2]:
train_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_train_XY_preprocesado.csv'
df_train_so = pd.read_csv(train_XY_url)

In [3]:
test_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_test_XY_preprocesado.csv'
df_test_XY = pd.read_csv(test_XY_url)

In [4]:
train_outlier_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_train_XY_preprocesado_y_outlier.csv'
df_train_XY = pd.read_csv(train_outlier_XY_url)

In [5]:
df_train_XY = df_train_XY.drop(['año_publicacion', 'property_coverage_percent'], axis=1)

In [6]:
df_validation_XY = df_train_XY.sample(frac=0.2)
df_validation_XY_copy = df_validation_XY.copy()
df_train_XY_copy = df_train_XY.copy()
df_test_XY_copy = df_test_XY.copy()

In [7]:
RANDOM_STATE = 3

## Preprocesamiento plus

In [8]:
from sklearn.preprocessing import OneHotEncoder

enc_language = OneHotEncoder()
X_t = df_train_XY[['property_type']]
X_t = enc_language.fit_transform(X_t).toarray()
language_columns = enc_language.get_feature_names_out()
#agrego las columnas al df_train
df_train_XY[language_columns] = X_t

df_train_XY = df_train_XY.drop(['property_type'], axis=1)

#repito el mismo proceso para df_validation
enc_language_val = OneHotEncoder()
X_v = df_validation_XY.loc[:,['property_type']]
X_v = enc_language_val.fit_transform(X_v).toarray()
language_columns_val = enc_language_val.get_feature_names_out()
df_validation_XY[language_columns_val] = X_v
df_validation_XY = df_validation_XY.drop(['property_type'], axis=1)

#repito el mismo proceso para df_test_XY
enc_language_val = OneHotEncoder()
X_v = df_test_XY.loc[:,['property_type']]
X_v = enc_language_val.fit_transform(X_v).toarray()
language_columns_val = enc_language_val.get_feature_names_out()
df_test_XY[language_columns_val] = X_v
df_test_XY = df_test_XY.drop(['property_type'], axis=1)


In [9]:
df_train_XY = df_train_XY.drop(['property_title'], axis=1)
df_validation_XY = df_validation_XY.drop(['property_title'], axis=1)
df_test_XY = df_test_XY.drop(['property_title'], axis=1)

In [10]:
df_train_XY = df_train_XY.drop(['place_l3'], axis=1)
df_validation_XY = df_validation_XY.drop(['place_l3'], axis=1)
df_test_XY = df_test_XY.drop(['place_l3'], axis=1)

In [11]:
df_train_XY = df_train_XY.drop(['start_date', 'end_date'], axis=1)
df_validation_XY = df_validation_XY.drop(['start_date', 'end_date'], axis=1)
df_test_XY = df_test_XY.drop(['start_date', 'end_date'], axis=1)

In [12]:
df_validation_Y = df_validation_XY['property_price']
df_validation_X = df_validation_XY.drop(['property_price'], axis=1)
df_train_Y = df_train_XY['property_price']
df_train_X = df_train_XY.drop(['property_price'], axis=1)
df_test_Y = df_test_XY['property_price']
df_test_X = df_test_XY.drop(['property_price'], axis=1)

## Buscamos Hiper parámetros con Normalización usamos GridSearchCV

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [14]:
# Definir los hiperparámetros a buscar
param_grid = {
    'kernel': ['rbf'],
    'C': [1, 10],
    'epsilon': [0.1, 0.2]
}

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
# Normalizar los datos de entrada
scaler = StandardScaler()
df_train_X_normalizado = scaler.fit_transform(df_train_X)
df_validation_X_normalizado = scaler.transform(df_validation_X)
df_test_X_normalizado = scaler.transform(df_test_X)

In [17]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# Definir el modelo de SVM de regresión
model = SVR()

# Cantidad de splits para el Cross Validation
folds = 2

# Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

# Métrica

scorer_fn = make_scorer(mean_absolute_error, greater_is_better=False)

# GridSearch CV
gridcv = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scorer_fn, cv=kfoldcv, return_train_score=True, verbose=10)

# Mostrar los resultados de la búsqueda
gridcv.fit(df_train_X_normalizado, df_train_Y)

gridcv.cv_results_


Fitting 2 folds for each of 4 candidates, totalling 8 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


[CV 1/2; 1/4] START C=1, epsilon=0.1, kernel=rbf................................
[CV 1/2; 1/4] END C=1, epsilon=0.1, kernel=rbf;, score=(train=-104489.999, test=-104178.440) total time= 2.0min
[CV 2/2; 1/4] START C=1, epsilon=0.1, kernel=rbf................................
[CV 2/2; 1/4] END C=1, epsilon=0.1, kernel=rbf;, score=(train=-104169.842, test=-104482.234) total time= 1.9min
[CV 1/2; 2/4] START C=1, epsilon=0.2, kernel=rbf................................
[CV 1/2; 2/4] END C=1, epsilon=0.2, kernel=rbf;, score=(train=-104489.999, test=-104178.440) total time= 1.9min
[CV 2/2; 2/4] START C=1, epsilon=0.2, kernel=rbf................................
[CV 2/2; 2/4] END C=1, epsilon=0.2, kernel=rbf;, score=(train=-104169.842, test=-104482.234) total time= 1.9min
[CV 1/2; 3/4] START C=10, epsilon=0.1, kernel=rbf...............................
[CV 1/2; 3/4] END C=10, epsilon=0.1, kernel=rbf;, score=(train=-92471.041, test=-92142.145) total time= 1.9min
[CV 2/2; 3/4] START C=10, epsilon=0.

{'mean_fit_time': array([64.03024983, 62.74374497, 60.39312196, 60.93965232]),
 'std_fit_time': array([1.78867102, 0.70056331, 0.15431762, 0.70363224]),
 'mean_score_time': array([53.54106009, 54.0247575 , 53.68062997, 54.47880828]),
 'std_score_time': array([0.41143835, 0.79567468, 0.04343867, 0.95107138]),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epsilon': masked_array(data=[0.1, 0.2, 0.1, 0.2],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'rbf', 'rbf', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'epsilon': 0.1, 'kernel': 'rbf'},
  {'C': 1, 'epsilon': 0.2, 'kernel': 'rbf'},
  {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'},
  {'C': 10, 'epsilon': 0.2, 'kernel': 'rbf'}],
 'split0_test_score': arr

In [18]:
best_model = gridcv.best_estimator_

In [19]:
gridcv.best_estimator_

SVR(C=10)

In [22]:
gridcv.best_params_

{'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}

In [23]:
# Realizar predicciones en el conjunto de entrenamiento
pred_train_y = best_model.predict(df_train_X_normalizado)
# Realizar predicciones en el conjunto de prueba con el mejor modelo
pred_val_y = best_model.predict(df_validation_X_normalizado)

In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [25]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_train = mean_squared_error(df_train_Y, pred_train_y)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_train = mean_absolute_error(df_train_Y, pred_train_y)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_train = np.sqrt(mse_train)

# Calcular el error cuadrático medio (MSE) en los datos de prueba
mse_val = mean_squared_error(df_validation_Y, pred_val_y)

# Calcular el error absoluto medio (MAE) en los datos de prueba
mae_val = mean_absolute_error(df_validation_Y, pred_val_y)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de prueba
rmse_val = np.sqrt(mse_val)

# Comparar los errores de entrenamiento y prueba
print("Error cuadrático medio (MSE) - Entrenamiento:", mse_train)
print("Error cuadrático medio (MSE) - Prueba:", mse_val)
print("Error absoluto medio (MAE) - Entrenamiento:", mae_train)
print("Error absoluto medio (MAE) - Prueba:", mae_val)
print("Raíz del error cuadrático medio (RMSE) - Entrenamiento:", rmse_train)
print("Raíz del error cuadrático medio (RMSE) - Prueba:", rmse_val)

Error cuadrático medio (MSE) - Entrenamiento: 35737017864.7711
Error cuadrático medio (MSE) - Prueba: 35319047272.65273
Error absoluto medio (MAE) - Entrenamiento: 85086.15818877425
Error absoluto medio (MAE) - Prueba: 84344.6167079897
Raíz del error cuadrático medio (RMSE) - Entrenamiento: 189042.37055425195
Raíz del error cuadrático medio (RMSE) - Prueba: 187933.6246461839


## Probamos en test

In [26]:
pred_test_y = best_model.predict(df_test_X_normalizado)

In [27]:
# Calcular el error cuadrático medio (MSE) en los datos de prueba
mse_test = mean_squared_error(df_validation_Y, pred_val_y)

# Calcular el error absoluto medio (MAE) en los datos de prueba
mae_test = mean_absolute_error(df_validation_Y, pred_val_y)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de prueba
rmse_test = np.sqrt(mse_test)

# Errores test
print("Error cuadrático medio (MSE) - Test:", mse_test)
print("Error absoluto medio (MAE) - Test:", mae_test)
print("Raíz del error cuadrático medio (RMSE) - Test:", rmse_test)

Error cuadrático medio (MSE) - Test: 35319047272.65273
Error absoluto medio (MAE) - Test: 84344.6167079897
Raíz del error cuadrático medio (RMSE) - Test: 187933.6246461839


In [28]:
import pickle

# Guardar el modelo entrenado en un archivo
with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [29]:
# Cargar el modelo desde el archivo
with open('modelo_entrenado.pkl', 'rb') as file:
    loaded_model_pickle = pickle.load(file)

In [30]:
# Realizar predicciones en el conjunto de prueba con el modelo cargado con pickle
pred_test_y_loaded_pickle = loaded_model_pickle.predict(df_test_X_normalizado)

In [31]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_test_bis = mean_squared_error(df_test_Y, pred_test_y_loaded_pickle)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_test_bis = mean_absolute_error(df_test_Y, pred_test_y_loaded_pickle)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_test_bis = np.sqrt(mse_test_bis)

# Errores test
print("Error cuadrático medio (MSE) - Test:", mse_test_bis)
print("Error absoluto medio (MAE) - Test:", mae_test_bis)
print("Raíz del error cuadrático medio (RMSE) - Test:", rmse_test_bis)

Error cuadrático medio (MSE) - Test: 118018429367.68002
Error absoluto medio (MAE) - Test: 108868.08011904269
Raíz del error cuadrático medio (RMSE) - Test: 343538.1046808054
